In [21]:
# Import libraries

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import numpy as np
import math
import matplotlib

from numpy import random 
from numpy import linalg as LA
from scipy.optimize import least_squares

## Steps to use widgets
##-----------------------------
## To install ipympl with conda:
#conda install -c conda-forge ipympl
## If using the Notebook
#conda install -c conda-forge widgetsnbextension
## If using JupyterLab
#conda install nodejs
#jupyter labextension install @jupyter-widgets/jupyterlab-manager
#jupyter labextension install jupyter-matplotlib

## To install ipympl with pip (I did this):
#pip install ipympl
##-----------------------------

# To activate widgets (requires ipympl)
#%matplotlib widget
%matplotlib widget
import matplotlib.pyplot as plt

# Reduce print precision to make it easier to compare numbers
np.set_printoptions(precision=3)


In [22]:
# Functions
# --------------------------------------

# Simple generic function: y = w0+w1(x1)+w2(x2)+w3(x1x2)+w4(x1^2)+w5(x2^2)
def yfunction(X, w):
    rt2 = np.sqrt(2)
    y = w[0] + (w[1] * X[:,0] * rt2) + (w[2] * X[:,1] * rt2) + (w[3] * (X[:,0] * X[:,1]) * rt2) + \
    (w[4] * (X[:,0] * X[:,0])) + (w[5] * (X[:,1] * X[:,1]))
    return y

# Loss function: 1/n \sum_{i=1}^n (yi- f(x1i,x2i))^2 + lambda ||W||_2^2
def lossfunction(w, y, X, lambdaval):
    n = len(X[:,0])
    normW = LA.norm(w)
    fx1x2 = yfunction(X, w)
    loss = LA.norm(y - fx1x2)
    loss = np.divide(1.0, n) * (loss * loss) + (lambdaval * normW * normW) 
    return loss   

# --------------------------------------

# Function to build the matrix varphi(X)=[x1 x2 x1*x2 x1*x1 x2*x2]
def varphifunction(X,disp=True):
    rt2 = np.sqrt(2)
    varphi = np.zeros((len(X[:,0]), 6))
    varphi[:,0] = np.ones(len(X[:,0]))
    varphi[:,1] = rt2*X[:,0]
    varphi[:,2] = rt2*X[:,1]
    varphi[:,3] = rt2*np.multiply(X[:,0],X[:,1])
    varphi[:,4] = np.multiply(X[:,0],X[:,0])
    varphi[:,5] = np.multiply(X[:,1],X[:,1])
    if disp:
        print('The matrix varphi(X)=[1 sqrt(2)*x1 sqrt(2)x2 sqrt(2)*x1*x2 x1*x1 x2*x2]:')
        print(varphi)
        print('-----------------')
    return varphi

# Function to solve directly from the lifted space
def directfunction(w, y, X, lambdaval,disp=False):
    n = len(X[:,0])
    varphi = varphifunction(X,disp=disp)
    varphi_transp = np.transpose(varphi)
    lambdaI = n * lambdaval * np.identity(len(w))
    xmatrix = np.dot(varphi_transp, varphi)
    xmatrix = xmatrix + lambdaI
    yvector = np.dot(varphi_transp,y)
    w = LA.solve(xmatrix,yvector)
    if disp:
        print('lambda=',lambdaval)
        print('-----------------')
        print('The matrix varphi(X)^T:')
        print(varphi_transp)
        print('-----------------')
        print('The matrix n*lambda*I:')
        print(lambdaI)
        print('-----------------')
        print('The vector varphi(X)^T*y:')
        print(yvector)
        print('-----------------')
    return w

# --------------------------------------

# Function to construct the kernel matrix K_i,j = (<varphi(x_i),varphi(x_j)>+1)^2 for the quadratic problem
def kernelfunction(X,disp=False):
    if disp:
        print('X=[x1 x2] is:')
        print(X)
        print('-----------------')
    # Form the matrix in a loop for demonstration purposes
    K = np.zeros(shape=(len(X[:,0]),len(X[:,0])))
    sizes = K.shape
    for i in range(0,sizes[0]):
        for j in range(0,sizes[1]):
            K[i,j] = np.power(np.dot(X[i,:],X[j,:]) + 1,2)
    if disp:
        print('The kernel matrix K:')
        print(K)
        print('-----------------')
        print('The size of K is ', sizes)
    return K

# Function to solve for the coefficients for the kernel matrix method
def kernelmethodfunction(y, X, lambdaval,disp=False):
    if disp:
        print('lambda=',lambdaval)
        print('-----------------')
    K = kernelfunction(X,disp=disp)
    n = len(X[:,0])
    lambdaI = lambdaval * np.identity(n) * n
    a = LA.solve(K+lambdaI,y)
    return a

# Function to evaluate y=Ka, using only the training data
def evalwithkernelfunction(a, X):
    K = kernelfunction(X)
    y = np.dot(K, a)
    return y

# Function to evaluate y=Ka, using a new test point xstar
def evalpointwithkernelfunction(a, xstar, X):
    # Make the projection vector k(x*,X)
    k = np.zeros((len(X[:,0]),1))
    for i in range(0,len(X[:,0])):
        k[i] = np.power(np.dot(xstar,X[i,:]) + 1,2)
    y = np.dot(np.transpose(k), a)
    return y

# --------------------------------------

In [23]:
# Generate some data 
# --------------------------------------

# Choose ten normally distributed random points for x1 and x2
npnts = 10
X = np.random.uniform(size=(npnts,2),high=1,low=-1)

In [24]:
X

array([[-0.154,  0.568],
       [ 0.118, -0.155],
       [ 0.299,  0.495],
       [-0.31 ,  0.936],
       [-0.871, -0.451],
       [ 0.87 ,  0.697],
       [ 0.025, -0.794],
       [ 0.306, -0.296],
       [-0.851,  0.222],
       [ 0.806,  0.061]])

In [25]:

# Create a polynomial, p(x1,x2) = \sum_{i,j} c_{i,j} * x1^i * x2^j
coeffs = [[1,1,1,1,1],[2,2,2,2,2]]
y_poly = np.polynomial.polynomial.polyval2d(X[:,0],X[:,1],coeffs)

In [26]:
# Add some white noise to the polynomial
noise = random.normal(0, 1.0, npnts)
noise_intensity = 100
y_poly = y_poly + np.divide(noise, noise_intensity)

In [27]:

# Initial coefficients for yfunction
w0 = np.array([1,1,1,1,1,1])

print('This is the data we will use')
print('=================')
print('X data:')
print(X)
print('-----------------')

print('y data:')
print(y_poly)
print('-----------------')

# The matrix varphi(X) = [1 x1 x2 x1*x2 x1*x1 x2*x2]
varphi = varphifunction(X)

This is the data we will use
X data:
[[-0.154  0.568]
 [ 0.118 -0.155]
 [ 0.299  0.495]
 [-0.31   0.936]
 [-0.871 -0.451]
 [ 0.87   0.697]
 [ 0.025 -0.794]
 [ 0.306 -0.296]
 [-0.851  0.222]
 [ 0.806  0.061]]
-----------------
y data:
[ 1.488  1.069  3.063  1.674 -0.521  7.559  0.779  1.238 -0.893  2.782]
-----------------
The matrix varphi(X)=[1 sqrt(2)*x1 sqrt(2)x2 sqrt(2)*x1*x2 x1*x1 x2*x2]:
[[ 1.000e+00 -2.172e-01  8.031e-01 -1.233e-01  2.359e-02  3.224e-01]
 [ 1.000e+00  1.662e-01 -2.198e-01 -2.583e-02  1.381e-02  2.417e-02]
 [ 1.000e+00  4.229e-01  7.000e-01  2.093e-01  8.941e-02  2.450e-01]
 [ 1.000e+00 -4.390e-01  1.324e+00 -4.111e-01  9.637e-02  8.769e-01]
 [ 1.000e+00 -1.231e+00 -6.380e-01  5.554e-01  7.578e-01  2.035e-01]
 [ 1.000e+00  1.230e+00  9.858e-01  8.574e-01  7.566e-01  4.859e-01]
 [ 1.000e+00  3.595e-02 -1.123e+00 -2.855e-02  6.463e-04  6.305e-01]
 [ 1.000e+00  4.322e-01 -4.184e-01 -1.279e-01  9.340e-02  8.751e-02]
 [ 1.000e+00 -1.204e+00  3.135e-01 -2.668e-01  7.24

In [28]:
# Solve using least squares
# --------------------------------------

print('First solve using least-squares loss with Tikonhov regularization:')
print('=================')

# Regularization parameter lambda
lambdaval = 1
print('lambda=',lambdaval)
print('-----------------')

res_1 = least_squares(lossfunction, w0, args=(y_poly, X, lambdaval))
w_ls = res_1.x
print('w_ls=',w_ls)
print('-----------------')

# Solve with smaller lambda
lambdaval2 = 0.01
res_2 = least_squares(lossfunction, w0, args=(y_poly, X, lambdaval2))
w_ls2 = res_2.x


First solve using least-squares loss with Tikonhov regularization:
lambda= 1
-----------------
w_ls= [0.735 0.852 0.559 0.394 0.318 0.323]
-----------------


In [29]:
# Solve using direct method
# --------------------------------------

print('Now solve using lifted space:')
print('=================')
w_lifted = directfunction(w0, y_poly, X, lambdaval,disp=True)
print('w_lifted=',w_lifted)
print('-----------------')

# Solve with smaller lambda
w_lifted2 = directfunction(w0, y_poly, X, lambdaval2,disp=False)


Now solve using lifted space:
The matrix varphi(X)=[1 sqrt(2)*x1 sqrt(2)x2 sqrt(2)*x1*x2 x1*x1 x2*x2]:
[[ 1.000e+00 -2.172e-01  8.031e-01 -1.233e-01  2.359e-02  3.224e-01]
 [ 1.000e+00  1.662e-01 -2.198e-01 -2.583e-02  1.381e-02  2.417e-02]
 [ 1.000e+00  4.229e-01  7.000e-01  2.093e-01  8.941e-02  2.450e-01]
 [ 1.000e+00 -4.390e-01  1.324e+00 -4.111e-01  9.637e-02  8.769e-01]
 [ 1.000e+00 -1.231e+00 -6.380e-01  5.554e-01  7.578e-01  2.035e-01]
 [ 1.000e+00  1.230e+00  9.858e-01  8.574e-01  7.566e-01  4.859e-01]
 [ 1.000e+00  3.595e-02 -1.123e+00 -2.855e-02  6.463e-04  6.305e-01]
 [ 1.000e+00  4.322e-01 -4.184e-01 -1.279e-01  9.340e-02  8.751e-02]
 [ 1.000e+00 -1.204e+00  3.135e-01 -2.668e-01  7.245e-01  4.913e-02]
 [ 1.000e+00  1.140e+00  8.624e-02  6.949e-02  6.493e-01  3.718e-03]]
-----------------
lambda= 1
-----------------
The matrix varphi(X)^T:
[[ 1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00  1.000e+00
   1.000e+00  1.000e+00  1.000e+00  1.000e+00]
 [-2.172e-01  1.662e-

In [30]:
# Solve using kernel method
# --------------------------------------

print('Now solve using kernel matrix:')
print('=================')
a_km = kernelmethodfunction(y_poly, X, lambdaval)
print('a_km=',a_km)
print('-----------------')

# Solve with smaller lambda
a_km2 = kernelmethodfunction(y_poly, X, lambdaval2)


Now solve using kernel matrix:
a_km= [ 0.043  0.031  0.139  0.042 -0.037  0.449  0.045  0.036 -0.092  0.079]
-----------------


In [31]:
# Coefficient comparison
# --------------------------------------

# Compare coefficients for direct and kernel methods
print('Method | lambda | coefficients, w')
print('---------------------------------')
print('DS     | 1.0    | '+str(w_ls))
print('LS     | 1.0    | '+str(w_lifted))
#print('LS     | 0.1    | ',w_ls2)
#print('ES     | 0.1    | ',w_es2)
print('')
print('Kernel | lambda | coefficients, a')
print('---------------------------------')
print('KER   | 1.0    | '+str(a_km))
#print('Quad   | 0.1    | ',a_km2)

# Direct comparison of the kernel matrix coefficients and the expanded space ones
print('')
print('Kernel | lambda | coefficients, w vs varphi(X)^T a')
print('--------------------------------------------------')
print('ES     | 1.0    | '+str(w_lifted))
print('KER    | 1.0    | '+str(np.dot(a_km,varphi)))
#print('ES     | 0.1    | ',w_es2)
#print('Quad   | 0.1    | ',np.dot(a_km2,varphi))

Method | lambda | coefficients, w
---------------------------------
DS     | 1.0    | [0.735 0.852 0.559 0.394 0.318 0.323]
LS     | 1.0    | [0.735 0.852 0.559 0.394 0.318 0.323]

Kernel | lambda | coefficients, a
---------------------------------
KER   | 1.0    | [ 0.043  0.031  0.139  0.042 -0.037  0.449  0.045  0.036 -0.092  0.079]

Kernel | lambda | coefficients, w vs varphi(X)^T a
--------------------------------------------------
ES     | 1.0    | [0.735 0.852 0.559 0.394 0.318 0.323]
KER    | 1.0    | [0.735 0.852 0.559 0.394 0.318 0.323]


In [32]:
# Pointwise comparison
# --------------------------------------

# Dot product
y_ls_line = varphi @ w_ls
y_ls_line2 = varphi @ w_ls2
y_lifted_line = varphi @ w_lifted
y_lifted_line2 = varphi @ w_lifted2

# Evaluate kernel with coefficients a_km
y_km_line = evalwithkernelfunction(a_km, X)
y_km_line2 = evalwithkernelfunction(a_km2, X)


In [33]:
# Compare the three solution methods
# ----------------------------------

# big lambda
x1,x2=X[:,0],X[:,1]
# Generate a grid of points to plot the surface on
inc = 0.1
x1_pnts = np.arange(min(min(x1),min(x2)), max(max(x1),max(x2))+inc, inc)
x2_pnts = np.arange(min(min(x1),min(x2)), max(max(x1),max(x2))+inc, inc)
x1_grid, x2_grid = np.meshgrid(x1_pnts, x2_pnts, sparse=False, indexing='ij')
sizes=x1_grid.shape

# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_ls, color="red", linewidth=0, antialiased=False, alpha=0.3)
surf = ax.plot_surface(x1_grid, x2_grid, y_es, color="blue", linewidth=0, antialiased=False, alpha=0.3)
surf = ax.plot_surface(x1_grid, x2_grid, y_km, color="green", linewidth=0, antialiased=False, alpha=0.3)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for all fits vs data with $\lambda = 1.0$')
ax.grid()
start, end = ax.get_xlim()
plt.show()

# small lambda

# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_ls2, color="red", linewidth=0, antialiased=False, alpha=0.3)
surf = ax.plot_surface(x1_grid, x2_grid, y_es2, color="blue", linewidth=0, antialiased=False, alpha=0.3)
surf = ax.plot_surface(x1_grid, x2_grid, y_km2, color="green", linewidth=0, antialiased=False, alpha=0.3)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for all fits vs data with $\lambda = 0.1$')
ax.grid()
start, end = ax.get_xlim()
plt.show()

FigureCanvasNbAgg()

ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [34]:
# Plot the comparison for least squares
# -------------------------------------
x1,x2=X[:,0],X[:,1]
# Generate a grid of points to plot the surface on
inc = 0.1
x1_pnts = np.arange(min(min(x1),min(x2)), max(max(x1),max(x2))+inc, inc)
x2_pnts = np.arange(min(min(x1),min(x2)), max(max(x1),max(x2))+inc, inc)
x1_grid, x2_grid = np.meshgrid(x1_pnts, x2_pnts, sparse=False, indexing='ij')
sizes=x1_grid.shape

# Initialise solution vectors
y_ls = np.zeros((sizes[0],sizes[1]))
y_ls2 = np.zeros((sizes[0],sizes[1]))

for i in range(0,sizes[0]):
    for j in range(0,sizes[1]):
        # Generate y_ls from the least squares coefficients (w_ls)
        y_ls[i,j] = yfunction(np.column_stack([x1_grid[i,j], x2_grid[i,j]]), w_ls)
        # Generate y_ls2 from the least squares coefficients (w_ls2)
        y_ls2[i,j] = yfunction(np.column_stack([x1_grid[i,j], x2_grid[i,j]]), w_ls2)

# big lambda
        
# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_ls, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for least squares fit vs data with $\lambda=$ %.2f' %lambdaval)
ax.grid()
start, end = ax.get_xlim()
plt.show()

# small lambda

# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_ls2, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for least squares fit vs data with $\lambda=$ %.2f' %lambdaval2)
ax.grid()
start, end = ax.get_xlim()
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [35]:
# Plot the comparison for direct solve of lifted space
# ------------------------------------------------------

# Initialise solution vectors
y_es = np.zeros((sizes[0],sizes[1]))
y_es2 = np.zeros((sizes[0],sizes[1]))

for i in range(0,sizes[0]):
    for j in range(0,sizes[1]):
        # Generate y_es from the lifted space coefficients (w_es)
        y_es[i,j] = yfunction(np.column_stack([x1_grid[i,j], x2_grid[i,j]]), w_lifted)
        # Generate y_es2 from the lifted space coefficients (w_es2)
        y_es2[i,j] = yfunction(np.column_stack([x1_grid[i,j], x2_grid[i,j]]), w_lifted2)
        
# big lambda
        
# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_es, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for lifted space fit vs data with $\lambda=$ %.2f' %lambdaval)
ax.grid()
start, end = ax.get_xlim()
plt.show()

# small lambda

# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_es2, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for lifted space fit vs data with $\lambda=$ %.2f' %lambdaval2)
ax.grid()
start, end = ax.get_xlim()
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [36]:
# Generate data to compare for kernel matrix method
# -------------------------------------------------

Xstar = [0,0]

# Initialise solution vectors
y_km = np.zeros((sizes[0],sizes[1]))
y_km2 = np.zeros((sizes[0],sizes[1]))

for i in range(0,sizes[0]):
    for j in range(0,sizes[1]):
        # Xstar=[x1 x2] value at corrent grid point
        Xstar[0] = x1_grid[i,j]
        Xstar[1] = x2_grid[i,j]
        # Generate y_km from the kernel method coefficients (w_km)
        y_km[i,j] = evalpointwithkernelfunction(a_km, Xstar, X)
        # Generate y_km2 from the kernel method coefficients (w_km2)
        y_km2[i,j] = evalpointwithkernelfunction(a_km2, Xstar,X)

In [17]:
a_km

array([ 0.045,  0.014,  0.046, -0.25 ,  0.57 ,  0.027, -0.043, -0.005,
       -0.047,  0.282])

In [18]:
# Plot the comparison for kernel matrix method
# --------------------------------------------

# big lambda

# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_km, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for kernel matrix method fit vs data with $\lambda=$ %.2f' %lambdaval)
ax.grid()
start, end = ax.get_xlim()
plt.show()
        
# small lambda
    
# Plot the surface and draw on the data (x1, x2, y_poly)
fig = plt.figure()
ax = fig.gca(projection='3d')
surf = ax.plot_surface(x1_grid, x2_grid, y_km2, cmap=cm.coolwarm, linewidth=0, antialiased=False, alpha=0.7)
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
fig.colorbar(surf, shrink=0.5, aspect=5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Surface for kernel matrix method fit vs data with $\lambda=$ %.2f' %lambdaval2)
ax.grid()
start, end = ax.get_xlim()
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [19]:
# Plot the comparison
# -------------------

# big lambda
x1,x2 = X[:,0],X[:,1]
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
ax.plot(x1, x2, y_ls_line, linewidth=0, marker="P", color="red", alpha=0.5)
ax.plot(x1, x2, y_lifted_line, linewidth=0, marker="o", color="blue", alpha=0.5)
ax.plot(x1, x2, y_km_line, linewidth=0, marker="p", color="green", alpha=0.5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Fits vs data for $\lambda=$ %.2f' %lambdaval)
ax.legend(('Data','LS','Lifted','KM'))
ax.grid()
start, end = ax.get_xlim()
plt.show()

# small lambda

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(x1, x2, y_poly, linewidth=0, marker="x", color="black")
ax.plot(x1, x2, y_ls_line2, linewidth=0, marker="P", color="red", alpha=0.5)
ax.plot(x1, x2, y_lifted_line2, linewidth=0, marker="o", color="blue", alpha=0.5)
ax.plot(x1, x2, y_km_line2, linewidth=0, marker="p", color="green", alpha=0.5)
ax.set(xlabel=r'$x_1$', ylabel=r'$x_2$', zlabel=r'$y(x_1,x_2)$', title=r'Fits vs data for $\lambda=$ %.2f' %lambdaval2)
ax.legend(('Data','LS','Lifted','KM'))
ax.grid()
start, end = ax.get_xlim()

# This might not be necessary - it seemed like there were too many ticks at one stage
# stepsize=0.25
# ax.xaxis.set_ticks(np.arange(start, end, stepsize))
# ax.yaxis.set_ticks(np.arange(start, end, stepsize))

plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()